# LangChain Toolbox + MCP (Restaurant Demo)

This notebook shows how to run a local MCP server and call its tools with the Toolbox LangChain SDK and langchain-openai.


## Setup (uv init + venv)

```bash
uv init
uv venv
.\.venv\Scripts\activate
uv add mcp toolbox-langchain langchain-openai langgraph python-dotenv
```

Create a `.env` file in this folder:

```bash
OPENAI_API_KEY=...
TOOLBOX_URL=http://127.0.0.1:5000
ADMIN_TOKEN=admin-token
```


## Start the MCP server

Run this in a separate terminal:

```bash
python server.py
```

The server listens on http://127.0.0.1:5000.


In [ ]:
from dotenv import load_dotenv

load_dotenv()


## Load tools (toolset)


In [ ]:
from toolbox_langchain import ToolboxClient

TOOLBOX_URL = "http://127.0.0.1:5000"

headers = {
    "Accept": "application/json, text/event-stream",
}
# https://github.com/modelcontextprotocol/python-sdk/issues/1641

with ToolboxClient(TOOLBOX_URL, client_headers=headers) as toolbox:
    tools = toolbox.load_toolset()
    print([t.name for t in tools])


In [ ]:
async with ToolboxClient(TOOLBOX_URL) as toolbox:
    tools = await toolbox.aload_toolset()

[tool.name for tool in tools]


## Use case 1: manual tool calls


In [ ]:
async with ToolboxClient(TOOLBOX_URL) as toolbox:
    list_prices = toolbox.load_tool("list_pizza_prices")
    get_price = toolbox.load_tool("get_pizza_price")
    get_hours = toolbox.load_tool("get_opening_hours")

    prices = list_prices.invoke({})
    margherita = get_price.invoke({"pizza": "margherita"})
    saturday = get_hours.invoke({"day": "sat"})

prices, margherita, saturday


## Use case 3: LangChain + OpenAI (agent via LangGraph)

This uses a small ReAct-style agent that can call the MCP tools automatically.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

async with ToolboxClient(TOOLBOX_URL) as toolbox:
    tools = toolbox.load_toolset()
    agent = create_agent(model, tools)

    prompt = "What are the pizza prices and the opening hours on Saturday?"
    result = agent.invoke({"messages": [("user", prompt)]})
    agent_answer = result["messages"][-1].content

agent_answer


## Use case 4: LangChain + OpenAI (bind_tools)

This binds the MCP tools to the model and runs tool calls manually.


In [ ]:
from langchain_core.messages import ToolMessage

async with ToolboxClient(TOOLBOX_URL) as toolbox:
    tools = toolbox.load_toolset()
    tool_map = {tool.name: tool for tool in tools}
    model_with_tools = model.bind_tools(tools)

    prompt = "What are the pizza prices and the opening hours on Saturday?"
    first = model_with_tools.invoke(prompt)

    if not first.tool_calls:
        final_answer = first.content
    else:
        tool_messages = []
        for call in first.tool_calls:
            result = tool_map[call["name"]].invoke(call["args"])
            tool_messages.append(
                ToolMessage(content=str(result), tool_call_id=call["id"])
            )

        final = model_with_tools.invoke([("user", prompt), first, *tool_messages])
        final_answer = final.content

final_answer
